In [1]:
import sys
sys.path.append('C:/Users/felix/Documents/Studium Ingenieurinformatik/dAISY/fboehning')

In [2]:
import pandas as pd
import pyperclip

import importlib

import beesup_llm; importlib.reload(beesup_llm)
import beesup_llm.toolkit.setup_utils; importlib.reload(beesup_llm.toolkit.setup_utils); from beesup_llm.toolkit.setup_utils import *
import beesup_llm.toolkit.visualization; importlib.reload(beesup_llm.toolkit.visualization); from beesup_llm.toolkit.visualization import *
import beesup_llm.toolkit.display_settings; importlib.reload(beesup_llm.toolkit.display_settings); from beesup_llm.toolkit.display_settings import *


ModuleNotFoundError: No module named 'pyperclip'

In [ ]:
knowledge_df = pd.read_pickle('datasets/0006_dataset/dataset_df.pkl')
knowledge_df

# QDQ - Query Driven Questions

- Attribute-Based Understanding
- test the ability to connect and contextualize information from disparate chunks into a unified understanding.
- only cover knowlege with attr_type=='Attribute'
- Assesses categorization skills by identifying entities that fall into predefined groups based on their attributes.
- the answers cannot be derived from a single chunk but involve synthesizing information from several chunks
- "SQL-like" answer


In [5]:
# get attrs_df
attrs_df=knowledge_df[knowledge_df['attr_type']=='Attribute'].copy()
attrs_df=attrs_df[['s','p','o']]
attrs_df.rename(
    columns={
        's':'scientific_name',
        'p':'attribute',
        'o':'value'
        },inplace=True)

attrs_df=attrs_df.pivot(index='scientific_name', columns='attribute', values='value').reset_index()
attrs_df.columns.name=None
attrs_df = attrs_df.reset_index(drop=True)
attrs_df.fillna('', inplace=True)
attrs_df

,scientific_name,Baumaterial,Familie,Kuckucksbienen,Kuckuckswespen,Nistweise,Pollensammelverhalten,Requisiten,Sozialverhalten,Umgangssprachliche Bezeichnungen,Wirtsbienen,Wissenschaftlicher Name
0,"Aglaoapis tridentata (Nylander, 1848)",,Megachilidae,,,,,,parasitisch,Dunkle Zweizahnbiene; Zweizahnbiene,"Hoplitis anthocopoides (Schenck, 1853); Hoplit...","Aglaoapis tridentata (Nylander, 1848)"
1,"Amegilla quadrifasciata (de Villers, 1789)",,Apidae,"Thyreus histrionicus (Illiger, 1806)",,endogäisch-selbstgegrabene Hohlräume,polylektisch,,,Weiße Bindenpelzbiene,,"Amegilla quadrifasciata (de Villers, 1789)"
2,"Ammobates punctatus (Fabricius, 1804)",,Apidae,,,,,,parasitisch,Große Sandgängerbiene; Sandgänger-Biene,"Anthophora bimaculata (Panzer, 1798)","Ammobates punctatus (Fabricius, 1804)"
3,"Ammobatoides abdominalis (Eversmann, 1852)",,,,,,,,,Steppenglanzbiene,,
4,"Andrena afzeliella (Kirby, 1802)",,Andrenidae,,,endogäisch-selbstgegrabene Hohlräume,,,,Kleine Kleesandbiene; Kleine Kleesandbiene,,"Andrena afzeliella (Kirby, 1802)"
...,...,...,...,...,...,...,...,...,...,...,...,...
506,"Thyreus orbatus (Lepeletier, 1841)",,Apidae,,,,,,parasitisch,Fleckenbiene; Schwarzgesichtige Fleckenbiene,"Anthophora borealis Morawitz, 1864; Anthophora...","Thyreus orbatus (Lepeletier, 1841)"
507,"Trachusa byssina (Panzer, 1798)","Blattstücke, Harz",Megachilidae,"Coelioxys conica (Linnaeus, 1758)",,endogäisch-selbstgegrabene Hohlräume,oligolektisch,,,Große Harzbiene,,"Trachusa byssina (Panzer, 1798)"
508,"Xylocopa iris (Christ, 1791)",Mark,Apidae,,,hypergäisch-selbstgegrabene Hohlräume,polylektisch,Pflanzenstängel,,Kleine Holzbiene,,"Xylocopa iris (Christ, 1791)"
509,"Xylocopa valga (Gerstäcker, 1872)",Holzgenagsel,,,Polochrum repandum Spinola 1806,hypergäisch-selbstgegrabene Hohlräume,,Totholz,,Schwarzfühler-Holzbiene; Südliche Holzbiene; Ö...,,


In [8]:
# Query Driven Questions
QDQ_SAMPLES=[
dict(
    conditions=(
        (attrs_df.Familie=='Megachilidae')
        &(attrs_df.Pollensammelverhalten.str.contains('polylektisch'))
        &(attrs_df['Baumaterial'].str.contains('Blattstücke'))
    ),
    target_attrs=['scientific_name'],
    question='Welche Bienenarten der Megachilidae sammelen Pollen von vielen verschiedenen Pflanzenarten und bauen ihre Nester aus Blattstücken?'
),
dict(
    conditions=(
        (attrs_df.Kuckucksbienen.str.contains('Nomada panzeri'))  
    ),
    target_attrs=['Familie'],
    question='Für die Arten welcher Bienenfamilie ist die Art Nomada panzeri eine Kuckucksbiene?'
),
dict(
    conditions=(
        (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Zwergsandbiene')) 
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche unterschiedlichen Arten der Zwergsandbiene gibt es?'
),
dict(
    conditions=(
        (attrs_df.Familie=='Halictidae')
        & (attrs_df['Nistweise'].str.contains('endogäisch'))
        & (attrs_df['Pollensammelverhalten'].str.contains('oligolektisch'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten der Familie Halictidae bauen ihre Nester in der Erde und sammeln Pollen von wenigen Pflanzenarten?'
),
dict(
    conditions=(
        (attrs_df['Nistweise'].str.contains('hypergäisch'))
        & (attrs_df['Baumaterial'].str.contains('Harz'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten bauen ihre Nester oberirdisch und verwenden Harz als Baumaterial?'
),
dict(
    conditions=(
        (attrs_df.Familie=='Halictidae')
        & (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Furchenbiene'))
        & (attrs_df.Pollensammelverhalten=='polylektisch') 
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten der Familie Halictidae werden als Furchenbienen bezeichnet und sammeln Pollen von vielen verschiedenen Pflanzenarten?'
),
dict(
    conditions=(
        (attrs_df['scientific_name'].str.contains('Morawitz'))
        & (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Wespenbiene'))  
    ),
    target_attrs=['Umgangssprachliche Bezeichnungen'],
    question='Welche Wespenbienen wurden durch Morawitz beschrieben?'
),
dict(
    conditions=(
        (attrs_df.Familie=='Halictidae')
        & (attrs_df['Wirtsbienen'].str.contains('Andrena'))       
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten der Familie Halictidae parasitieren die Gattung Andrena?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('komunal'))  
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten haben ein komunales Sozialverhalten?'
),
dict(
    conditions=(
        (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Filzbiene'))  
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten werden als Filzbienen bezeichnet?'
),
dict(
    conditions=(
        (attrs_df['Kuckuckswespen'].str.contains('Sapyga'))
        & (attrs_df.Pollensammelverhalten.str.contains('polylektisch')) 
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten sammeln Pollen von vielen verschiedenen Pflanzenarten und haben Kuckuckswespen der Gattung Sapyga?'
),
dict(
    conditions=(
        (attrs_df['Kuckuckswespen'].str.contains('Sapyga'))
        & (attrs_df.Pollensammelverhalten.str.contains('oligolektisch')) 
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten sammeln Pollen von wenigen Pflanzenarten und haben Kuckuckswespen der Gattung Sapyga?'
),
dict(
    conditions=(
        (attrs_df['Baumaterial'].str.contains('Lehm'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten verwenden unter anderem Lehm als Baumaterial?'
),
dict(
    conditions=(
        (attrs_df['Nistweise'].str.contains('Insektenfraßgänge'))
        &(attrs_df['Baumaterial'].str.contains('Blattstücke'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten bauen ihre Nester in Insektenfraßgängen und verwenden Blattstücke als Baumaterial?'
),
dict(
    conditions=(
        (attrs_df['Nistweise'].str.contains('Mauerfugen'))
        &(attrs_df.Pollensammelverhalten.str.contains('polylektisch'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten sammeln Pollen von vielen verschiedenen Pflanzenarten und bauen ihre Nester in Mauerfugen?'
),
dict(
    conditions=(
        (attrs_df.Kuckucksbienen.str.contains('Aglaoapis'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten haben Bienen der Art Aglaoapis als Kuckucksbienen?'
),
dict(
    conditions=(
        (attrs_df.Familie=='Colletidae')
        &(attrs_df['Requisiten'].str.contains('Sand'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten der Familie Colletidae verwenden Sand als Requisiten?'
),
dict(
    conditions=(
        (attrs_df['Requisiten'].str.contains('Schnecken'))
        &(attrs_df.Kuckucksbienen.str.contains('Stelis odontopyga'))

    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten bauen ihre Nester in Schneckenhäusern und haben Bienen der Art Stelis odontopyga als Kuckucksbienen?'
),
dict(
    conditions=(

        (attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Maskenbiene'))
        &(attrs_df.Pollensammelverhalten.str.contains('breit-oligolektisch'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Maskenbienen zeigen ein breit-oligolektisches Pollensammelverhalten?'
),
dict(
    conditions=(
        (attrs_df['Sozialverhalten'].str.contains('solitär'))
        &(attrs_df['Nistweise'].str.contains('vorhanden'))
    ),
    target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
    question='Welche Bienenarten haben ein solitäres Sozialverhalten und nutzen bereits vorhandene Nistplätze?'
),
# dict(
#     conditions=(
#         #(attrs_df.Familie=='Colletidae')
#         #(attrs_df['scientific_name'].str.contains('Morawitz'))
#         #(attrs_df['Umgangssprachliche Bezeichnungen'].str.contains('Wollbiene'))
#         #(attrs_df['Kuckuckswespen'].str.contains('Sapyga'))
#         (attrs_df['Sozialverhalten'].str.contains('solitär'))
#         #& (attrs_df['Wirtsbienen'].str.contains('Andrena'))
#         &(attrs_df['Nistweise'].str.contains('vorhanden'))
#         #&(attrs_df['Baumaterial'].str.contains('Blattstücke'))
#         #(attrs_df['Requisiten'].str.contains('Schnecken'))
#         #&(attrs_df.Kuckucksbienen.str.contains('Stelis odontopyga'))
#         #&(attrs_df.Pollensammelverhalten.str.contains('breit-oligolektisch'))
#         #& (attrs_df.Baumaterial=='Blattstücke')  
#     ),
#     target_attrs=['scientific_name','Umgangssprachliche Bezeichnungen'],
#     question=''
# ),
]

print(len(QDQ_SAMPLES))
attrs_df[QDQ_SAMPLES[-1]['conditions']]

20


,scientific_name,Baumaterial,Familie,Kuckucksbienen,Kuckuckswespen,Nistweise,Pollensammelverhalten,Requisiten,Sozialverhalten,Umgangssprachliche Bezeichnungen,Wirtsbienen,Wissenschaftlicher Name
249,"Hylaeus duckei (Alfken, 1904)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,breit-oligolektisch (Apiaceae),,solitär,Duckes Maskenbiene,,"Hylaeus duckei (Alfken, 1904)"
253,"Hylaeus hyalinatus (Smith, 1842)",,Colletidae,,,"hypergäisch-vorhandene Hohlräume, Mauerfugen, ...",polylektisch (15 Familien),,solitär,Mauer-Maskenbiene,,"Hylaeus hyalinatus (Smith, 1842)"
260,"Hylaeus paulus (Bridwell, 1919)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,polylektisch (starke Präf. Rosacaceae),,solitär,Kleine Maskenbiene,,"Hylaeus paulus (Bridwell, 1919)"
262,"Hylaeus pfankuchi (Alfken, 1919)",,Colletidae,,,"hypergäisch-vorhandene Hohlräume, Gallen?",polylektisch (starke Präf. Rosacaceae),(Schilfgallen?),solitär,Ried-Maskenbiene,,"Hylaeus pfankuchi (Alfken, 1919)"
264,"Hylaeus punctatus (Brullé, 1832)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,polylektisch (starke Präf. Apiaceae),,solitär,Grobpunktierte Maskenbiene,,"Hylaeus punctatus (Brullé, 1832)"
265,"Hylaeus punctulatissimus (Smith, 1842)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,eng-oligolektisch (Allium),,solitär,Lauch-Maskenbiene,,"Hylaeus punctulatissimus (Smith, 1842)"
270,"Hylaeus taeniolatus (Förster, 1871)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,breit-oligolektisch (Apiaceae),,solitär,Gelbhals-Maskenbiene,,"Hylaeus taeniolatus (Förster, 1871)"
271,"Hylaeus tyrolensis (Förster, 1871)",,Colletidae,,,hypergäisch-vorhandene Hohlräume,breit-oligolektisch (Apiaceae),,solitär,Tiroler Maskenbiene,,"Hylaeus tyrolensis (Förster, 1871)"
272,"Hylaeus variegatus (Fabricius, 1798)",,Colletidae,,,endogäisch-vorhandene Hohlräume,polylektisch (starke Präf. Apiaceae),,solitär,Rote Maskenbiene,,"Hylaeus variegatus (Fabricius, 1798)"


In [ ]:
#leaf trough the samples
qdq_sample=QDQ_SAMPLES.pop(0)
print(f"question: {qdq_sample['question']}")
#print(f"target_attrs: {qdq_sample['target_attrs']}")
attrs_df[qdq_sample['conditions']][qdq_sample['target_attrs']]


question: Welche Bienenarten der Megachilidae sammelen Pollen von vielen verschiedenen Pflanzenarten und bauen ihre Nester aus Blattstücken?


,scientific_name
340,"Megachile alpicola (Alfken, 1924)"
342,"Megachile apicalis (Spinola, 1808)"
343,"Megachile argentata (Fabricius, 1793)"
345,"Megachile centuncularis (Linnaeus, 1758)"
346,"Megachile circumcincta (Kirby, 1802)"
348,"Megachile genalis (Morawitz, 1880)"
349,"Megachile lagopoda (Linnaeus, 1761)"
351,"Megachile leachella (Curtis, 1828)"
352,"Megachile ligniseca (Kirby, 1802)"
353,"Megachile maritima (Kirby, 1802)"


# FFQ - Free Form Questions
- General Understanding/ General Abilities because an Answertext must be formulated

In [90]:
def compose_emb(row):
    s,p,o,so,po,spo=[row[f'{c}_emb'] for c in ['s','p','o','so','po','spo']]
    emb=po
    emb_normalized=emb/ np.linalg.norm(emb) #normalize

    compose_emb.info='po'
    return emb_normalized

passages_df=knowledge_df[knowledge_df['attr_type']=='Text'].copy()
passages_df=passages_df[['s','p','o']]
passages_df['emb']=knowledge_df.apply(compose_emb, axis=1)
passages_df.rename(
    columns={
        's':'scientific_name',
        'p':'attribute',
        'o':'passage'
        },inplace=True)
passages_df.reset_index(names='kid', inplace=True)
passages_df

,kid,scientific_name,attribute,passage,emb
0,0,"Aglaoapis tridentata (Nylander, 1848)",Aussehen,"Die Biene ist schwarz, etwa 8–10 mm lang und m...","[0.013315118, -0.0025690868, -0.002520601, -0...."
1,2,"Aglaoapis tridentata (Nylander, 1848)",Kurzbeschreibung,"Dunkle Zweizahnbiene (Aglaoapis tridentata, Sy...","[0.0050212676, -0.020270795, 0.03047962, -0.01..."
2,3,"Aglaoapis tridentata (Nylander, 1848)",Lebensraum,"Die Biene lebt auf trockenen, blütenreichen Bö...","[0.010719753, 0.0073655825, 0.023964873, -0.00..."
3,6,"Aglaoapis tridentata (Nylander, 1848)",Verbreitung,Die Biene kommt hauptsächlichen in Süd- und Mi...,"[0.018203076, -0.0093616815, 0.010161171, -0.0..."
4,11,"Amegilla quadrifasciata (de Villers, 1789)",Kurzbeschreibung,Amegilla quadrifasciata ist eine Biene aus der...,"[-0.005874074, 0.014487559, 0.025344735, -0.00..."
...,...,...,...,...,...
1039,3870,"Xylocopa iris (Christ, 1791)",Vorkommen,Die Art ist im Mittelmeerraum und östlich bis ...,"[0.018833201, 0.009829856, 0.026238477, 0.0120..."
1040,3880,"Xylocopa violacea (Linnaeus, 1758)",Kurzbeschreibung,"Die Große Blaue Holzbiene, auch Blauschwarze o...","[0.013091193, 0.0038780614, 0.022316568, 0.006..."
1041,3881,"Xylocopa violacea (Linnaeus, 1758)",Lebensweise,"Die Paarung erfolgt im Frühjahr, nachdem beide...","[-0.000986353, -0.0118575245, 0.024509192, 0.0..."
1042,3882,"Xylocopa violacea (Linnaeus, 1758)",Merkmale,Mit 20 bis 28 Millimeter Körperlänge zählen di...,"[0.013986376, 0.007970083, -0.004052453, -0.01..."


In [70]:
from sklearn.metrics.pairwise import cosine_similarity

passages_embs=np.vstack(passages_df['emb'].values)
match_matrix=cosine_similarity(passages_embs,passages_embs)
match_matrix.shape

(1044, 1044)

In [89]:
num_chunks=2

passage_row=passages_df.sample(1).iloc[0]

print(f"# {passage_row['attribute']}/ {passage_row['scientific_name']}\n")
print(f"{passage_row['passage']}")

match_index=passage_row.name
match_indices=match_matrix[match_index].argsort()[::-1][:num_chunks]
match_scores=match_matrix[match_index][match_indices]

ranking_df=passages_df.iloc[match_indices].copy()
ranking_df['score']=match_scores
ranking_df


# Merkmale/ Nomada obtusifrons (Nylander, 1848)

Die Bienen haben eine Körperlänge von fünf bis sieben Millimetern. Der Kopf und Thorax der Weibchen ist schwarz und ist rot gezeichnet. Letzterer hat manchmal auch kleine gelbliche Flecken. Die Tergite sind rot. Teilweise sind sie stark verdunkelt, teilweise haben sie weißgelbe Flecken. Das Labrum ist schwarz und hat ein kleines Zähnchen. Charakteristisch für die Art ist der abgeflachte Wulst zwischen den Fühlern, der bei ähnlichen Arten als kantiger Stirnkiel ausgebildet ist. Das Schildchen (Scutellum) ist schwarz und trägt deutliche Höcker. Die Schienen (Tibien) der Hinterbeine sind am Ende stumpf und haben vier oder fünf dicke, kurze, kleine Dornen. Die Männchen sind den Weibchen ähnlich, ihre Mandibeln und der Vorderrand der Stirnplatte sind jedoch gelb. Die Hintertibien tragen zwei bis drei kleine Dornen.


,kid,scientific_name,attribute,passage,emb,score
835,3144,"Nomada obtusifrons (Nylander, 1848)",Merkmale,Die Bienen haben eine Körperlänge von fünf bis...,"[0.015013425, -0.007370205, -0.0056248168, -0....",1.000000
853,3191,"Nomada rhenana (Morawitz, 1872)",Merkmale,Die Bienen haben eine Körperlänge von 7 bis 9 ...,"[0.019856127, -0.0008634387, -0.005509735, -0....",0.972185


In [ ]:
def get_ffq_prompt(df):

    if len(df)==1: chunk_str="chunk"
    else: chunk_str="chunks"

    prompt=""
    prompt+=f"""
You are a teacher creating evaluation questions for a student. Based on the knowledge {chunk_str} provided below, your task is to:
1. Generate a question in German that evaluates the student's understanding or ability to infer based on the knowledge.
2. Provide a "teacher's solution" in German that represents the correct answer.

Return the output in the following format as a JSON string:
{{
    "question": "Here goes the generated question in German",
    "answer": "Here goes the teacher's solution in German"
}}
""".strip()
    
    prompt+=f"\n\nKNOWLEDGE {chunk_str.upper()}:\n"
    
    for _,row in df.iterrows():
        prompt+="\n# {attribute} von {scientific_name}\n{passage}\n".format(**row)
    
    prompt+="\n\nThe output must be a valid JSON string."
    
    return prompt


prompt=get_ffq_prompt(ranking_df[:2])
pyperclip.copy(prompt)
print(prompt)

You are a teacher creating evaluation questions for a student. Based on the knowledge chunks provided below, your task is to:
1. Generate a question in German that evaluates the student's understanding or ability to infer based on the knowledge.
2. Provide a "teacher's solution" in German that represents the correct answer.

Return the output in the following format as a JSON string:
{
    "question": "Here goes the generated question in German",
    "answer": "Here goes the teacher's solution in German"
}

KNOWLEDGE CHUNKS:

# Verbreitung von Anthidium montanum (Morawitz, 1865)
Die Art ist extrem selten. Sie kommt auf den Gebirgen Europas (auf den Pyrenäen, den Alpen, dem Schwarzwald, dem Erzgebirge, den Karpaten, dem Krimgebirge) und Zentralasiens vor. Im deutschsprachigen Raum fast ausschließlich in den Alpen. Es gibt einen aktuellen Nachweis aus der Schwäbischen Alb, sonst überwiegend historische Funde.

# Vorkommen von Xylocopa iris (Christ, 1791)
Die Art ist im Mittelmeerraum und

In [86]:
output={
    "question": "Welche Gemeinsamkeiten und Unterschiede gibt es in der Verbreitung von Anthidium montanum und Xylocopa iris? Begründen Sie Ihre Antwort anhand der gegebenen Informationen.",
    "answer": "Gemeinsamkeiten: Beide Arten sind selten und haben ein Verbreitungsgebiet, das teilweise in Europa und Zentralasien liegt. Unterschiede: Anthidium montanum kommt vor allem in Gebirgsregionen vor, während Xylocopa iris vor allem im Mittelmeerraum verbreitet ist. In Deutschland findet man Anthidium montanum fast nur in den Alpen und einen Nachweis in der Schwäbischen Alb, während Xylocopa iris nur in Baden-Württemberg vorkommt. Zusätzlich ist Xylocopa iris in der Schweiz stark gefährdet, während für Anthidium montanum keine solche Gefährdung erwähnt wird."
}


In [ ]:
ffq_sample=dict(
    df=ranking_df,
    compose_emb_info=compose_emb.info,
    prompt=prompt
    output=
)

ffq_sample

{'id': 2923,
 'scientific_name': 'Nomada errans (Lepeletier, 1841)',
 'header': 'Vorkommen und Lebensweise',
 'passage': 'Die Art ist in Süd- und mancherorts auch in Mitteleuropa und in England verbreitet. Die Tiere fliegen von Mitte Juni bis Mitte August. Die Art parasitiert Andrena nitidiuscula und vielleicht auch Andrena pallitarsis.'}

In [21]:
ffqs_df=pd.DataFrame()
ffqs_df

""


# MCQ - Multiple Choice Questions